# Data Model Explained

## Set up

In [61]:
import os
import glob
import psycopg2
import pandas as pd
from tabulate import tabulate

In [62]:
import warnings
warnings.filterwarnings('ignore')

In [63]:
import configparser

config = configparser.ConfigParser()
config.read_file(open('test.cfg'))

DB_NAME                = config.get("CLUSTER","DB_NAME")
DB_USER                = config.get("CLUSTER","DB_USER")
DB_PASSWORD            = config.get("CLUSTER","DB_PASSWORD")
DB_PORT                = config.get("CLUSTER","DB_PORT")
DWH_ENDPOINT           = config.get("CLUSTER","END_POINT")

conn_string="host={} dbname={} user={} password={} port={}".format(
    DWH_ENDPOINT,
    DB_NAME,
    DB_USER, 
    DB_PASSWORD, 
    DB_PORT
)

In [64]:
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [65]:
# export reddit db tables
from sqlalchemy_schemadisplay import create_schema_graph
from sqlalchemy import MetaData

conn_str="postgresql://{}:{}@{}:{}/{}".format(
    DB_USER, 
    DB_PASSWORD, 
    DWH_ENDPOINT, 
    DB_PORT, 
    DB_NAME
)

# graph = create_schema_graph(metadata=MetaData(conn_str, schema='reddit'))
# graph.write_png('../imgs/reddit_data_model.png')

In [66]:
# functions
def describe(table_name):
    query = f"""
    select 
        column_name,
        data_type,
        case when character_maximum_length is not null
            then character_maximum_length
            else numeric_precision end as max_length,
        is_nullable
    from information_schema.columns
    where table_name = '{table_name}'
    order by ordinal_position;"""
    result = pd.read_sql(query, conn)
    print(tabulate(result, headers='keys', tablefmt='psql'))

def count_rows(table_name):
    query = f"SELECT COUNT(*) FROM reddit.{table_name}"
    result = pd.read_sql(query, conn)
    print(tabulate(result, headers='keys', tablefmt='psql'))
    
def count_rows_by(table_name, col):
    query = f"SELECT {col}, COUNT(*) FROM reddit.{table_name} GROUP BY 1 ORDER BY 1 ASC"
    result = pd.read_sql(query, conn)
    print(tabulate(result, headers='keys', tablefmt='psql'))
    
def limit5(table_name):
    query = f"SELECT * FROM {table_name} LIMIT 5"
    return(pd.read_sql(query, conn))

## Data Model

### Schema
For this project we are creating a schema named `reddit` in AWS Redshift, to store our data model. In the table below, one can see all the tables available in the schema.

In [67]:
q = """
SELECT table_name, table_type
FROM information_schema.tables
WHERE table_schema = 'reddit'
order by 1 asc;
"""
results = pd.read_sql(q, conn)
print(tabulate(results, headers='keys', tablefmt='psql'))

+----+---------------------+--------------+
|    | table_name          | table_type   |
|----+---------------------+--------------|
|  0 | creators_d          | BASE TABLE   |
|  1 | creators_snapshot   | BASE TABLE   |
|  2 | dwr_dates           | BASE TABLE   |
|  3 | posts_d             | BASE TABLE   |
|  4 | posts_snapshot      | BASE TABLE   |
|  5 | reddit_fact         | BASE TABLE   |
|  6 | reddit_logs         | BASE TABLE   |
|  7 | staging_dwr_dates   | BASE TABLE   |
|  8 | staging_reddit_logs | BASE TABLE   |
|  9 | subreddits_d        | BASE TABLE   |
| 10 | subreddits_snapshot | BASE TABLE   |
+----+---------------------+--------------+


### Tables (with relations)

<div>
<img src="https://raw.githubusercontent.com/dsavg/capstone-data-engineering-project/master/imgs/tables.png" width="500"/>
</div>

### Helper Tables (no relations)
![helper_tables](https://raw.githubusercontent.com/dsavg/capstone-data-engineering-project/master/imgs/helper_tables.png)

## Data source size

In [68]:
print('>> Table Name')
print('reddit.reddit_logs')
print()
print('>> Describe')
describe('reddit_logs')
print()
print('>> Row Count')
count_rows('reddit_logs')
print()
print('>> Row Count by date')
count_rows_by('reddit_logs', 'snapshot_date')

>> Table Name
reddit.reddit_logs

>> Describe
+----+-------------------------+-------------------+--------------+---------------+
|    | column_name             | data_type         |   max_length | is_nullable   |
|----+-------------------------+-------------------+--------------+---------------|
|  0 | author                  | character varying |          256 | YES           |
|  1 | author_fullname         | character varying |          256 | YES           |
|  2 | author_is_blocked       | boolean           |          nan | YES           |
|  3 | subreddit               | character varying |          256 | YES           |
|  4 | subreddit_id            | character varying |          256 | YES           |
|  5 | subreddit_name_prefixed | character varying |          256 | YES           |
|  6 | subreddit_subscribers   | bigint            |           64 | YES           |
|  7 | subreddit_type          | character varying |          256 | YES           |
|  8 | created_utc            

In [69]:
limit5('reddit.reddit_logs')

,author,author_fullname,author_is_blocked,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_subscribers,subreddit_type,created_utc,domain,...,pinned,saved,score,send_replies,title,total_awards_received,ups,upvote_ratio,url,snapshot_date
0,MediterraneanSeal,t2_49c7vs3t,False,pettyrevenge,t5_2vg7t,r/pettyrevenge,3233531,public,1.672001e+09,self.pettyrevenge,...,False,False,530,True,You really enjoy being an jerk to my colleague...,1,530,0.98,https://www.reddit.com/r/pettyrevenge/comments...,2022-12-19
1,smolandtuff,t2_hrpxxwbh,False,TalesFromRetail,t5_2t2zt,r/TalesFromRetail,915476,public,1.667956e+09,self.TalesFromRetail,...,False,False,285,True,“Somebody keep eyes on the man with a knife!”,1,285,0.97,https://www.reddit.com/r/TalesFromRetail/comme...,2022-12-19
2,Eudonidano,t2_qtsma,False,aww,t5_2qh1o,r/aww,33074037,public,1.672252e+09,i.redd.it,...,False,False,65,True,This is the best photo I've ever taken. Meet Lou!,0,65,0.95,https://i.redd.it/cicxs4gi3q8a1.jpg,2022-12-19
3,ShadowspiritGamez787,t2_605nd5dy,False,mildlyinfuriating,t5_2ubgg,r/mildlyinfuriating,5257133,public,1.672253e+09,i.redd.it,...,False,False,140,True,Am I the only one who has this happen to them ...,0,140,0.88,https://i.redd.it/0ux37dl57q8a1.jpg,2022-12-19
4,Alvesartt_,t2_846t8zvk,False,creepy,t5_2raed,r/creepy,14697780,public,1.671451e+09,i.redd.it,...,False,False,628,True,👽(oc),1,628,0.89,https://i.redd.it/m4zk4ccpyv6a1.png,2022-12-19


In [70]:
print('>> Table Name')
print('reddit.dwr_dates')
print()
print('>> Describe')
describe('dwr_dates')
print()
print('>> Row Count')
count_rows('dwr_dates')

>> Table Name
reddit.dwr_dates

>> Describe
+----+---------------------------+-------------------+--------------+---------------+
|    | column_name               | data_type         |   max_length | is_nullable   |
|----+---------------------------+-------------------+--------------+---------------|
|  0 | date                      | date              |          nan | NO            |
|  1 | day                       | bigint            |           64 | YES           |
|  2 | day_name                  | character varying |          256 | YES           |
|  3 | week                      | bigint            |           64 | YES           |
|  4 | weekday                   | bigint            |           64 | YES           |
|  5 | month                     | bigint            |           64 | YES           |
|  6 | month_name                | character varying |          256 | YES           |
|  7 | year                      | bigint            |           64 | YES           |
|  8 | uni

In [71]:
limit5('reddit.dwr_dates')

,date,day,day_name,week,weekday,month,month_name,year,united_state_holidays,is_united_state_holiday,united_kingdom_holidays,is_united_kingdom_holiday,canada_holidays,is_canada_holiday,australia_holidays,is_australia_holiday,germany_holidays,is_germany_holiday
0,2022-12-01,1,Thursday,48,3,12,December,2022,,False,,False,,False,,False,,False
1,2022-12-02,2,Friday,48,4,12,December,2022,,False,,False,,False,,False,,False
2,2022-12-03,3,Saturday,48,5,12,December,2022,,False,,False,,False,,False,,False
3,2022-12-04,4,Sunday,48,6,12,December,2022,,False,,False,,False,,False,,False
4,2022-12-05,5,Monday,49,0,12,December,2022,,False,,False,,False,,False,,False


## Data Dictionary

### reddit.creators_d

In [72]:
print('>> Table Name')
print('reddit.creators_d')
print()
print('>> Describe')
describe('creators_d')
print('>> Row Count')
count_rows('creators_d')

>> Table Name
reddit.creators_d

>> Describe
+----+----------------+-------------------+--------------+---------------+
|    | column_name    | data_type         |   max_length | is_nullable   |
|----+----------------+-------------------+--------------+---------------|
|  0 | creator_id     | character varying |          256 | NO            |
|  1 | object_type_id | character varying |          256 | YES           |
|  2 | name           | character varying |          256 | YES           |
|  3 | is_blocked     | boolean           |          nan | YES           |
|  4 | is_active      | boolean           |          nan | YES           |
+----+----------------+-------------------+--------------+---------------+
>> Row Count
+----+---------+
|    |   count |
|----+---------|
|  0 |   36965 |
+----+---------+


In [73]:
limit5('reddit.creators_d')

,creator_id,object_type_id,name,is_blocked,is_active
0,t2_1004is,t2,jfarag90,False,True
1,t2_10058u,t2,jaire96,False,True
2,t2_100b3rsm,t2,LilTacticalOnReddit,False,True
3,t2_100h1k,t2,Stas9t,False,True
4,t2_100ml6,t2,Winnebago01,False,True


### reddit.creators_snapshot

In [74]:
print('>> Table Name')
print('reddit.creators_snapshot')
print()
print('>> Describe')
describe('creators_snapshot')
print()
print('>> Row Count')
count_rows('creators_snapshot')
print()
print('>> Row Count by date')
count_rows_by('creators_snapshot', 'snapshot_date')

>> Table Name
reddit.creators_snapshot

>> Describe
+----+----------------+-------------------+--------------+---------------+
|    | column_name    | data_type         |   max_length | is_nullable   |
|----+----------------+-------------------+--------------+---------------|
|  0 | creator_id     | character varying |          256 | NO            |
|  1 | object_type_id | character varying |          256 | NO            |
|  2 | name           | character varying |          256 | NO            |
|  3 | is_blocked     | boolean           |          nan | NO            |
|  4 | is_active      | boolean           |          nan | NO            |
|  5 | snapshot_date  | character varying |          256 | YES           |
+----+----------------+-------------------+--------------+---------------+

>> Row Count
+----+---------+
|    |   count |
|----+---------|
|  0 |  285160 |
+----+---------+

>> Row Count by date
+----+-----------------+---------+
|    | snapshot_date   |   count |
|----+-

In [75]:
limit5('reddit.creators_snapshot')

,creator_id,object_type_id,name,is_blocked,is_active,snapshot_date
0,t2_z8c3tk,t2,smuglips,False,True,2022-12-21
1,t2_v5izhlvu,t2,Son-of-Prophet,False,True,2022-12-21
2,t2_gmd16xbi,t2,Beagle_bango,False,True,2022-12-21
3,t2_wpw75,t2,MythicalZelda,False,True,2022-12-21
4,t2_i28ew35c,t2,Ecstatic-Sun9265,False,True,2022-12-21


### reddit.subreddits_d

In [76]:
print('>> Table Name')
print('reddit.subreddits_d')
print()
print('>> Describe')
describe('subreddits_d')
print()
print('>> Row Count')
count_rows('subreddits_d')

>> Table Name
reddit.subreddits_d

>> Describe
+----+-----------------+-------------------+--------------+---------------+
|    | column_name     | data_type         |   max_length | is_nullable   |
|----+-----------------+-------------------+--------------+---------------|
|  0 | subreddit_id    | character varying |          256 | NO            |
|  1 | name            | character varying |          256 | YES           |
|  2 | object_type_id  | character varying |          256 | YES           |
|  3 | prefixed_name   | character varying |          256 | YES           |
|  4 | type            | character varying |          256 | YES           |
|  5 | num_subscribers | bigint            |           64 | YES           |
+----+-----------------+-------------------+--------------+---------------+

>> Row Count
+----+---------+
|    |   count |
|----+---------|
|  0 |     182 |
+----+---------+


In [77]:
limit5('reddit.subreddits_d')

,subreddit_id,name,object_type_id,prefixed_name,type,num_subscribers
0,t5_2cneq,politics,t5,r/politics,public,8265625
1,t5_2fwo,programming,t5,r/programming,public,5025485
2,t5_2mx48p,madeinpython,t5,r/madeinpython,public,16593
3,t5_2qh03,gaming,t5,r/gaming,public,35671976
4,t5_2qh0f,entertainment,t5,r/entertainment,public,3606562


### reddit.subreddits_snapshot

In [78]:
print('>> Table Name')
print('reddit.subreddits_snapshot')
print()
print('>> Describe')
describe('subreddits_snapshot')
print()
print('>> Row Count')
count_rows('subreddits_snapshot')
print()
print('>> Row Count by date')
count_rows_by('subreddits_snapshot', 'snapshot_date')

>> Table Name
reddit.subreddits_snapshot

>> Describe
+----+-----------------+-------------------+--------------+---------------+
|    | column_name     | data_type         |   max_length | is_nullable   |
|----+-----------------+-------------------+--------------+---------------|
|  0 | subreddit_id    | character varying |          256 | NO            |
|  1 | name            | character varying |          256 | YES           |
|  2 | object_type_id  | character varying |          256 | YES           |
|  3 | prefixed_name   | character varying |          256 | YES           |
|  4 | type            | character varying |          256 | YES           |
|  5 | num_subscribers | bigint            |           64 | YES           |
|  6 | snapshot_date   | character varying |          256 | YES           |
+----+-----------------+-------------------+--------------+---------------+

>> Row Count
+----+---------+
|    |   count |
|----+---------|
|  0 |    3773 |
+----+---------+

>> Row Cou

In [79]:
limit5('reddit.subreddits_snapshot')

,subreddit_id,name,object_type_id,prefixed_name,type,num_subscribers,snapshot_date
0,t5_2qh1i,AskReddit,t5,r/AskReddit,public,39084824,2022-12-19
1,t5_2t2zt,TalesFromRetail,t5,r/TalesFromRetail,public,915476,2022-12-19
2,t5_2tz1e,MorbidReality,t5,r/MorbidReality,public,900782,2022-12-19
3,t5_2ubgg,mildlyinfuriating,t5,r/mildlyinfuriating,public,5257133,2022-12-19
4,t5_2ug83,creepyPMs,t5,r/creepyPMs,public,783140,2022-12-19


### reddit.posts_d

In [80]:
print('>> Table Name')
print('reddit.posts_d')
print()
print('>> Describe')
describe('posts_d')
print()
print('>> Row Count')
count_rows('posts_d')

>> Table Name
reddit.posts_d

>> Describe
+----+------------------------+-------------------+--------------+---------------+
|    | column_name            | data_type         |   max_length | is_nullable   |
|----+------------------------+-------------------+--------------+---------------|
|  0 | post_id                | character varying |          256 | NO            |
|  1 | title                  | character varying |          500 | YES           |
|  2 | object_type_id         | character varying |          256 | YES           |
|  3 | type                   | character varying |          256 | YES           |
|  4 | domain                 | character varying |          256 | YES           |
|  5 | url_link               | character varying |         2500 | YES           |
|  6 | hide_score             | boolean           |          nan | YES           |
|  7 | no_follow              | boolean           |          nan | YES           |
|  8 | is_created_from_ads_ui | boolean      

In [81]:
limit5('reddit.posts_d')

,post_id,title,object_type_id,type,domain,url_link,hide_score,no_follow,is_created_from_ads_ui,is_crosspostable,is_video,gilded,over_18,pinned
0,t3_2pk3li,"When I try to do squats, the bar hurts my neck...",t3,None,self.Fitness,https://www.reddit.com/r/Fitness/comments/2pk3...,False,False,False,True,False,0,False,False
1,t3_2qw7xh,Anyone else go to the gym at midnight?,t3,None,self.Fitness,https://www.reddit.com/r/Fitness/comments/2qw7...,False,False,False,True,False,0,False,False
2,t3_2soh9s,What is the best fluid to mix with Creatine?,t3,None,self.Fitness,https://www.reddit.com/r/Fitness/comments/2soh...,False,False,False,True,False,0,False,False
3,t3_2v5ygv,What is up with most gym's music?,t3,None,self.Fitness,https://www.reddit.com/r/Fitness/comments/2v5y...,False,False,False,True,False,0,False,False
4,t3_2vrm3t,A brief guide to shoulder impingement,t3,None,self.Fitness,https://www.reddit.com/r/Fitness/comments/2vrm...,False,False,False,True,False,0,False,False


### reddit.posts_snapshot

In [82]:
print('>> Table Name')
print('reddit.posts_snapshot')
print()
print('>> Describe')
describe('posts_snapshot')
print()
print('>> Row Count')
count_rows('posts_snapshot')
print()
print('>> Row Count by date')
count_rows_by('posts_snapshot', 'snapshot_date')

>> Table Name
reddit.posts_snapshot

>> Describe
+----+------------------------+-------------------+--------------+---------------+
|    | column_name            | data_type         |   max_length | is_nullable   |
|----+------------------------+-------------------+--------------+---------------|
|  0 | post_id                | character varying |          256 | NO            |
|  1 | title                  | character varying |          500 | YES           |
|  2 | object_type_id         | character varying |          256 | YES           |
|  3 | type                   | character varying |          256 | YES           |
|  4 | domain                 | character varying |          256 | YES           |
|  5 | url_link               | character varying |         2500 | YES           |
|  6 | hide_score             | boolean           |          nan | YES           |
|  7 | no_follow              | boolean           |          nan | YES           |
|  8 | is_created_from_ads_ui | boolea

In [83]:
limit5('reddit.posts_snapshot')

,post_id,title,object_type_id,type,domain,url_link,hide_score,no_follow,is_created_from_ads_ui,is_crosspostable,is_video,gilded,over_18,pinned,snapshot_date
0,t3_z20oaj,"""Where is the toy section?""",t3,None,self.TalesFromRetail,https://www.reddit.com/r/TalesFromRetail/comme...,False,False,False,True,False,0,False,False,2022-12-21
1,t3_zrpgd3,Forgot about this Gem,t3,None,i.redd.it,https://i.redd.it/sdn60zbu4b7a1.png,False,False,False,True,False,0,True,False,2022-12-21
2,t3_zu8gew,Best Christmas/winter themed episodes?,t3,None,self.gamegrumps,https://www.reddit.com/r/gamegrumps/comments/z...,False,False,False,True,False,0,False,False,2022-12-21
3,t3_zvl8cv,Has RTD inherited the show in better condition...,t3,None,i.redd.it,https://i.redd.it/my4pu24eq98a1.jpg,False,True,False,True,False,0,False,False,2022-12-21
4,t3_zvmd0i,"What if Disney decided to cancel the Simpsons,...",t3,None,self.TheSimpsons,https://www.reddit.com/r/TheSimpsons/comments/...,False,True,False,True,False,0,False,False,2022-12-21


### reddit.reddit_fact

In [84]:
print('>> Table Name')
print('reddit.reddit_fact')
print()
print('>> Describe')
describe('reddit_fact')
print()
print('>> Row Count')
count_rows('reddit_fact')
print()
print('>> Row Count by date')
count_rows_by('reddit_fact', 'dt')

>> Table Name
reddit.reddit_fact

>> Describe
+----+-----------------------+-----------------------------+--------------+---------------+
|    | column_name           | data_type                   |   max_length | is_nullable   |
|----+-----------------------+-----------------------------+--------------+---------------|
|  0 | event_id              | bigint                      |           64 | NO            |
|  1 | creator_id            | character varying           |          256 | NO            |
|  2 | subreddit_id          | character varying           |          256 | NO            |
|  3 | post_id               | character varying           |          256 | NO            |
|  4 | created_at            | date                        |          nan | NO            |
|  5 | created_at_timestamp  | timestamp without time zone |          nan | YES           |
|  6 | num_comments          | bigint                      |           64 | YES           |
|  7 | num_crossposts        | big

In [24]:
limit5('reddit.reddit_fact')

,event_id,creator_id,subreddit_id,post_id,created_at,created_at_timestamp,num_comments,num_crossposts,score,total_awards_received,ups,upvote_ratio,dt
0,42026,t2_rbhi0,t5_2qzb6,t3_z7s1gd,2022-11-29,2022-11-29 11:57:45,1,0,0,0,0,0.38,2022-12-21
1,42028,t2_1gah18in,t5_2t2zt,t3_z20oaj,2022-11-22,2022-11-22 18:18:06,52,0,591,0,591,0.99,2022-12-21
2,42030,t2_s7uif,t5_2snxj,t3_zvvkzz,2022-12-26,2022-12-26 20:07:28,5,0,27,0,27,0.89,2022-12-21
3,42032,t2_w2xb9,t5_2va9w,t3_ziv43j,2022-12-11,2022-12-11 14:54:45,111,0,1087,0,1087,0.88,2022-12-21
4,42034,t2_glpdmky9,t5_2ug83,t3_zrpgd3,2022-12-21,2022-12-21 15:01:54,4,0,51,0,51,0.96,2022-12-21
